### Notas del cuaderno de trabajo

El presente cuaderno de trabajo realiza casi lo mismo que el cuaderno 'clasificación_clientes.ipynb', con una sutiles diferencias:
- El presente cuaderno de trabajo normaliza cada día individualmente, con el fin de que días anómalos no aplanen los demas días (antes se normalizaba directamente los datos de todo el año)
- Ahora se usa la mediana para generar la curva tipo de cada cliente (es más resistente a valores atípicos)

Se ha implementado una función que genera todas las gráficas de curvas de todos los 'n' días que tiene un cliente, debido a que para cada cliente este proceso toma aproximadamente 80s, se ha decidido realizarlo solo para 20 clientes seleccionados:
Tenía pensado realizarlo para los siguientes clientes:
- 12
- 1234002
- 1457182
- 1627579
- 10020880
- 90000503
- 90001202
- TEXTILES TEXSA
- GC NOVOPAN DEL ECUADOR
- FLEXIPLAST
- ECOFROZ
- IDEAL ALAMBREC
- LOS COCOS
- PLANTA PANIFICADORA
- PRINTOPAC

En forma de array para copiar en código:
 - arr_clientes_graf = ['12', '1234002', '1457182', '1627579', '10020880', '90000503', '90001202', 'TEXTILES TEXSA', 'GC NOVOPAN DEL ECUADOR', 'FLEXIPLAST', 'ECOFROZ', 'IDEAL ALAMBREC', 'LOS COCOS', 'PLANTA PANIFICADORA', 'PRINTOPAC']

### Importación de librerías

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
mmscaler = MinMaxScaler()

from tqdm import tqdm
from sklearn.metrics import silhouette_score

import warnings
warnings.filterwarnings("ignore")

### Funciones para leer archivos .csv con diversos formatos

In [4]:
def intentar_abrir_archivo_datos(path_archivo):

    df_archivo_telem_tab = pd.DataFrame([1,2,3], columns=["prueba"])
    df_archivo_telem_pc = pd.DataFrame([1,2,3], columns=["prueba"])
    df_archivo_telem_c = pd.DataFrame([1,2,3], columns=["prueba"])

    # Intentar leer el archivo con separador 'tab'
    try:
        #print("Leyendo con tab")
        df_archivo_telem_tab = pd.read_csv(path_archivo,
                                            sep="\t",
                                            #decimal=",",
                                            skiprows=11,
                                            #na_values="N/D",
                                            encoding="utf-16",
                                            #on_bad_lines="skip",
                                            encoding_errors="ignore" 
                                            ) 
    except:
        pass

    # Intentar leer el archivo con separador 'punto y coma'
    try:
        #print("Leyendo con ;")
        df_archivo_telem_pc = pd.read_csv(path_archivo,
                                            sep=";",
                                            #decimal=",",
                                            skiprows=11,
                                            #na_values="N/D",
                                            #encoding="utf-16",
                                            #on_bad_lines="skip",
                                            encoding_errors="ignore" 
                                            )

    except:
        pass

    # Intentar leer el archivo con separador 'coma'
    try:
        #print("Leyendo con ,")
        df_archivo_telem_c = pd.read_csv(path_archivo,
                                            sep=",",
                                            #decimal=",",
                                            skiprows=11,
                                            #na_values="N/D",
                                            #encoding="utf-16",
                                            #on_bad_lines="skip",
                                            encoding_errors="ignore" 
                                            )

    except:
        pass



    if len(df_archivo_telem_tab.columns) > 1:
        df_archivo_telem = df_archivo_telem_tab.copy()
    elif len(df_archivo_telem_pc.columns) > 1:
        df_archivo_telem = df_archivo_telem_pc.copy()
    else:
        df_archivo_telem = df_archivo_telem_c.copy()

    return df_archivo_telem

### 1. Extracción de los archivos de cada grupo de clientes

#### 1.1 Extracción de los archivos con las mediciones mensuales del primer grupo de clientes

Estos clientes están clasificados por CUEN, y el formato de sus archivos varían, el procedimiento a seguir será:
1. Iterar sobre cada código de cliente
2. Buscar sus archivos de datos mensuales
3. Unificar todos en un solo archivo
4. El identificador a usar es el código de cliente

In [5]:
mediciones_clientes_g1 = r"../data/mediciones_por_mes_g1"
archivos_mediciones_g1 = list(os.scandir(mediciones_clientes_g1))
columnas_extraer_g1 = ["Fecha","Demanda activa DEL","Demanda reactiva DEL"]

In [6]:
clientes_unicos_g1 = set()

for medicion in archivos_mediciones_g1:
    cliente = medicion.name.split('-')[1]
    clientes_unicos_g1.add(cliente)

print(f"Clientes único encontrados en la carpeta de mediciones del grupo uno: {len(clientes_unicos_g1)}")

Clientes único encontrados en la carpeta de mediciones del grupo uno: 314


In [7]:
# Diccionario para almacenar los clientes con sus respectivos datos
dict_dfs_clientes_g1 = {}

# Iterar sobre cada cliente
for cliente in tqdm(clientes_unicos_g1, desc="Procesando clientes del grupo 01"):
    datos_cliente = []
    # Iterar sobre cada archivo de medicion del grupo 01
    for medicion in archivos_mediciones_g1:
        if cliente == medicion.name.split("-")[1]:
            df_cliente = intentar_abrir_archivo_datos(f"{mediciones_clientes_g1}/{medicion.name}")
            datos_cliente.extend(df_cliente[columnas_extraer_g1].values)

    # Convertir a DataFrame los datos concatenados
    df_datos_anual_cliente = pd.DataFrame(datos_cliente, columns=columnas_extraer_g1)
    
    # Almacenar en el diccionario (Clave->Cliente   Valor->DataFrame)
    dict_dfs_clientes_g1[cliente]=df_datos_anual_cliente
    
    # Eliminar dataframe concatenado para liberar memoria
    del df_datos_anual_cliente
    #df_datos_anual_cliente.to_csv(f"mediciones_por_anio/g1_perfil_carga_anual-{cliente}-2023.csv", index=False)

Procesando clientes del grupo 01: 100%|██████████| 314/314 [00:43<00:00,  7.26it/s]


#### 1.2 Extracción de los archivos con las mediciones mensuales del segundo grupo de clientes

De estos clientes tenemos carpetas con sus mediciones por mes, no existe tabla de excel inicial, se procederá a realizar lo siguiente:
1. Iterar sobre cada carpeta (cliente)
2. Obtener los datos de sus 12 meses
3. Unificar en un único archivo anual
4. Se usará el nombre del cliente como identificador

In [8]:
mediciones_clientes_g2 = "../data/mediciones_por_mes_g2"
archivos_mediciones_g2 = list(os.scandir(mediciones_clientes_g2))
columnas_extraer_g2 = ["Fecha", "AS (kWh)"]

In [9]:
print(f"Clientes único encontrados en la carpeta de mediciones del grupo uno: {len(archivos_mediciones_g2)}")

Clientes único encontrados en la carpeta de mediciones del grupo uno: 75


In [10]:
# Diccionario para almacenar los clientes con sus datos
dict_dfs_clientes_g2 = {}

# Iterar sobre cada cliente
for archivos_cliente in tqdm(archivos_mediciones_g2, desc="Procesando clientes del grupo 02"):
    nombre_cli = archivos_cliente.name.strip()
    df_concat = pd.DataFrame()

    # Obtener los archivos de las mediciones mensuales del cliente
    mediciones_mensuales_cliente = os.scandir(rf"{mediciones_clientes_g2}/{nombre_cli}")

    for medicion in mediciones_mensuales_cliente:
        medicion_mensual = pd.read_csv(rf"{mediciones_clientes_g2}/{nombre_cli}/{medicion.name}", sep=";", skiprows=2, encoding='ISO-8859-1')
        medicion_mensual = medicion_mensual[columnas_extraer_g2]
        df_concat = pd.concat([df_concat, medicion_mensual])

    # Almacenar en el diccionario (Clave->Cliente   Valor->DataFrame)
    dict_dfs_clientes_g2[nombre_cli] = df_concat
    
    # Eliminar dataframe concatenado para liberar memoria
    del df_concat
    #df_concat.to_csv(rf"mediciones_por_anio/g2_perfil_carga_anual-{nombre_cli}-2023.csv", index=False)

Procesando clientes del grupo 02: 100%|██████████| 75/75 [00:05<00:00, 13.75it/s]


### 2. Preprocesamiento de los datos

Ahora tenemos todos los datos unificados anualmente por cada cliente, tenemos que limpiar y preprocesar, realizar las siguientes tareas:
1. Calcular la potencia aparente (resultado de aplicar teorema pitágoras sobre potencia activa y reactiva)
2. Separar la columna 'fecha' en dos columnas 'fecha' y 'hora', fecha va a tener formato 'año/mes/dia' y hora el formato 'hh:mm'
3. Excluir aquellos registros que correspondan a fechas de sábado, domingo o días de feriado nacional
4. Normalizar los datos para que todos estén en la misma escala

In [11]:
feriados_nacionales = ["2/7/2023", "20/2/2023", "21/2/2023", "7/4/2023", "1/5/2023", \
                       "26/5/2023", "11/8/2023", "9/10/2023", "2/10/2023", "3/10/2023", "25/12/2023"]
feriados_nacionales = pd.to_datetime(feriados_nacionales, format='%d/%m/%Y')

dict_meses = {"01": "Enero",
              "02": "Febrero",
              "03": "Marzo",
              "04": "Abril",
              "05": "Mayo",
              "06": "Junio",
              "07": "Julio",
              "08": "Agosto",
              "09": "Septiembre",
              "10": "Octubre",
              "11": "Noviembre",
              "12": "Diciembre"}

print(list(dict_meses.keys()))
print(feriados_nacionales)

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
DatetimeIndex(['2023-07-02', '2023-02-20', '2023-02-21', '2023-04-07',
               '2023-05-01', '2023-05-26', '2023-08-11', '2023-10-09',
               '2023-10-02', '2023-10-03', '2023-12-25'],
              dtype='datetime64[ns]', freq=None)


In [12]:
def fecha_formato_unico(fecha_str):

    # Poner separador único el '/' y año únicamente 2023
    fecha_str = fecha_str.replace('-','/').replace('2024','2023')

    if len(fecha_str.split('/')[0]) == 4: # Cuando el formato es año/mes/día
        return fecha_str
    elif len(fecha_str.split('/')[0]) != 4: # Cuando el formato es día/mes/año
        seps = fecha_str.split('/')
        return f"{seps[-1]}/{seps[1]}/{seps[0]}"

2.1 - Transformación columna fecha archivos anuales grupo 01

In [13]:
dict_dfs_procesados_g1 = {}

for cliente, df_archivo_g1 in dict_dfs_clientes_g1.items():

    # Transformar columna fecha a cadena
    df_archivo_g1["Fecha"] = df_archivo_g1["Fecha"].astype("string")

    # Eliminar los valores nulos en la columna 'Fecha'
    df_archivo_g1 = df_archivo_g1.dropna(subset="Fecha")

    # Separar para obtener columna Hora
    df_archivo_g1["Hora"] = df_archivo_g1["Fecha"].apply(lambda x: x.split()[1].strip())

    # Separar para obtener columna Fecha
    df_archivo_g1["Fecha"] = df_archivo_g1["Fecha"].apply(lambda x: x.split()[0].strip())

    # Transformar la columna Fecha a un formato único
    df_archivo_g1["Fecha"] = df_archivo_g1["Fecha"].apply(fecha_formato_unico)

    # Debido a que a veces se ponen datos del 2024 para reemplazar los faltantes del 2023
    # debemos descartar la fecha 29 de febrero, pues en 2023 no existe
    df_archivo_g1 = df_archivo_g1[df_archivo_g1["Fecha"] != "2023/02/29"]
    
    # Eliminar duplicados y conservar el original del 2023
    df_archivo_g1["Fecha-Hora"] = df_archivo_g1["Fecha"] + " " + df_archivo_g1["Hora"]
    df_archivo_g1 = df_archivo_g1.drop_duplicates(subset="Fecha-Hora", keep="first")

    # Transformar columna Fecha a datetime
    df_archivo_g1["Fecha"] = pd.to_datetime(df_archivo_g1["Fecha"], format='%Y/%m/%d')

    # Obtener la potencia aparente
    df_archivo_g1["Potencia_aparente"] = np.sqrt((df_archivo_g1["Demanda activa DEL"]**2) + (df_archivo_g1["Demanda reactiva DEL"]**2))

    # Eliminar días feriados y días de fin de semana
    df_archivo_g1 = df_archivo_g1[~df_archivo_g1['Fecha'].isin(feriados_nacionales) & ~df_archivo_g1['Fecha'].dt.weekday.isin([5, 6])]

    # Interpolar valores nulos usando una función polinomial
    df_archivo_g1["Potencia_aparente"] = df_archivo_g1["Potencia_aparente"].interpolate(method='spline', order=3)

    # Conservar solo las columnas de interés
    df_archivo_g1 = df_archivo_g1[["Fecha", "Hora", "Potencia_aparente"]]

    # Escalar las mediciones (cada día se escala individualmente)
    df_archivo_g1["Potencia_aparente_escalada"] = df_archivo_g1.groupby("Fecha")["Potencia_aparente"].transform(
        lambda x: (x - x.min()) / (x.max() - x.min()) if (x.max() - x.min()) != 0 else 0
    )

    # Guardar en un nuevo diccionario los datos procesados
    dict_dfs_procesados_g1[cliente] = df_archivo_g1
    
    # Liberar memoria
    del df_archivo_g1
    
#del dict_dfs_clientes_g1

2.2 - Transformación columna fecha archivos anuales grupo 02

In [14]:
dict_dfs_procesados_g2 = {}

for cliente, df_archivo_g2 in dict_dfs_clientes_g2.items():

    # Transformar columna fecha a cadena
    df_archivo_g2["Fecha"] = df_archivo_g2["Fecha"].astype("string")

    # Eliminar registros que contienen el total
    df_archivo_g2 = df_archivo_g2[~df_archivo_g2["Fecha"].str.contains("Total")]

    # Eliminar los valores nulos en la columna 'Fecha'
    df_archivo_g2 = df_archivo_g2.dropna(subset="Fecha")

    # Separar para obtener columna Hora
    df_archivo_g2["Hora"] = df_archivo_g2["Fecha"].apply(lambda x: x.split()[1].strip())

    # Separar para obtener columna Fecha
    df_archivo_g2["Fecha"] = df_archivo_g2["Fecha"].apply(lambda x: x.split()[0].strip())

    # Eliminar duplicados y conservar el original del 2023
    df_archivo_g2["Fecha-Hora"] = df_archivo_g2["Fecha"] + " " + df_archivo_g2["Hora"]
    df_archivo_g2["Fecha-Hora"] = pd.to_datetime(df_archivo_g2["Fecha-Hora"], yearfirst=True)
    df_archivo_g2 = df_archivo_g2.drop_duplicates(subset="Fecha-Hora", keep="first")

    # Restar un timedelta de 15 a todas las fechas (Solo en este caso por que la ultima fecha)
    # se pasa al siguiente mes
    df_archivo_g2["Fecha-Hora"] = df_archivo_g2["Fecha-Hora"] - pd.Timedelta(minutes=15)

    # Separar nuevamente para obtener columna Hora
    df_archivo_g2["Hora"] = df_archivo_g2["Fecha-Hora"].astype("string").apply(lambda x: x.split()[1][:-3].strip())

    # Separar nuevamente para obtener columna Fecha
    df_archivo_g2["Fecha"] = df_archivo_g2["Fecha-Hora"].astype("string").apply(lambda x: x.split()[0].strip())

    # Transformar la columna Fecha a un formato único
    df_archivo_g2["Fecha"] = df_archivo_g2["Fecha"].apply(fecha_formato_unico)

    # Debido a que a veces se ponen datos del 2024 para reemplazar los faltantes del 2023
    # debemos descartar la fecha 29 de febrero, pues en 2023 no existe
    df_archivo_g2 = df_archivo_g2[df_archivo_g2["Fecha"] != "2023/02/29"]

    # Transformar columna Fecha a datetime
    df_archivo_g2["Fecha"] = pd.to_datetime(df_archivo_g2["Fecha"], format='%Y/%m/%d')

    # Limpiar la columna 'SE (KVah)'
    df_archivo_g2["AS (kWh)"] = df_archivo_g2["AS (kWh)"].astype("string").str.replace(",", "").replace('"','')
    df_archivo_g2["AS (kWh)"] = df_archivo_g2["AS (kWh)"].astype("float")

    # Obtener la potencia aparente
    df_archivo_g2["Potencia_aparente"] = df_archivo_g2["AS (kWh)"] * 4

    # Eliminar días feriados y días de fin de semana
    df_archivo_g2 = df_archivo_g2[~df_archivo_g2['Fecha'].isin(feriados_nacionales) & ~df_archivo_g2['Fecha'].dt.weekday.isin([5, 6])]

    # Interpolar valores nulos usando una función polinomial
    df_archivo_g2["Potencia_aparente"] = df_archivo_g2["Potencia_aparente"].interpolate(method='spline', order=3)

    # Conservar solo las columnas de interés
    df_archivo_g2 = df_archivo_g2[["Fecha", "Hora", "Potencia_aparente"]]

    # Escalar las mediciones (cada día se escala individualmente)
    df_archivo_g2["Potencia_aparente_escalada"] = df_archivo_g2.groupby("Fecha")["Potencia_aparente"].transform(
        lambda x: (x - x.min()) / (x.max() - x.min()) if (x.max() - x.min()) != 0 else 0
    )

    # Guardar en un nuevo diccionario los datos procesados
    dict_dfs_procesados_g2[cliente] = df_archivo_g2
    
    # Liberar memoria
    del df_archivo_g2
    
#del dict_dfs_clientes_g2

### 3. Generación de los entregables

In [15]:
def obtener_coords_curva_tipo(df):

    # Agrupar por hora y aplicar mediana
    df_grouped = df.groupby("Hora")["Potencia_aparente_escalada"].apply(np.median).sort_index(ascending=True).reset_index(drop=False)

    # Retornar el array con los 96 valores de demanda
    return df_grouped

def obtener_coords_dia_demanda_max(df):

    # Obtener el máximo valor de potencia aparente
    max_potencia = df['Potencia_aparente'].max()

    # Encontrar la fecha correspondiente a la máxima potencia aparente
    fecha_max_potencia = df[df['Potencia_aparente'] == max_potencia]['Fecha'].iloc[0]

    # Filtrar los registros correspondientes a esa fecha
    df_max_fecha = df[df['Fecha'] == fecha_max_potencia]

    # Ordenar el resultado de manera ascendente por 'Hora'
    df_max_fecha = df_max_fecha.sort_values(by="Hora", ascending=True)

    return fecha_max_potencia, df_max_fecha[["Hora","Potencia_aparente_escalada"]]

In [16]:
def graficar_curva_tipo(df, cod_cli, path, path_ctipos):

    # Generar el gráfico de la curva tipo
    _ = plt.figure(figsize=(16, 6))
    _ = plt.plot(df["Hora"], df["Potencia_aparente_escalada"], marker='o', color='b', linestyle='-', label='Potencia Aparente Escalada')
    _ = plt.title(f'Curva tipo cliente {cod_cli}')
    _ = plt.xlabel('Hora')
    _ = plt.ylabel('Potencia Aparente Escalada')
    _ = plt.grid(True)

    # Rotar etiquetas para que no se vea acumulado el eje X
    _ = plt.xticks(df["Hora"].values[::2], rotation=45)

    # Para que no se distorsione la dimenisión del eje Y
    _ = plt.yticks(np.arange(0, 1.1, 0.1))

    # Evitar recortes en las etiquetas
    _ = plt.tight_layout()

    # Guardar la gráfica en un directorio
    _ = plt.savefig(f"{path}/curva_tipo_{cod_cli}.png", format='png', dpi=150)  # Puedes cambiar el formato a 'jpg', 'pdf', etc.
    _ = plt.savefig(f"../outputs/{path_ctipos}/curva_tipo_{cod_cli}.png", format='png', dpi=150)  # Puedes cambiar el formato a 'jpg', 'pdf', etc.
    
    # Cerrar la figura después de guardarla para liberar recursos
    _ = plt.close()  

def graficar_dia_max_demanda(df, cod_cli, path, fecha):

    # Graficar la potencia aparente escalada a lo largo del día
    _ = plt.figure(figsize=(16, 6))
    _ = plt.plot(df["Hora"], df["Potencia_aparente_escalada"], marker='o', color='r', linestyle='-', label=f'Potencia Aparente Escalada')
    _ = plt.title(f'Curva del día de demanda máxima {fecha} para cliente {cod_cli}')
    _ = plt.xlabel('Hora')
    _ = plt.ylabel('Potencia Aparente Escalada')
    _ = plt.grid(True)
    _ = plt.legend()

    # Rotar etiquetas para que no se vea acumulado el eje X
    _ = plt.xticks(df["Hora"].values[::2], rotation=45)

    # Para que no se distorsione la dimenisión del eje Y
    _ = plt.yticks(np.arange(0, 1.1, 0.1))

    # Evitar recortes en las etiquetas
    _ = plt.tight_layout()

    # Guardar la gráfica en un archivo (por ejemplo, como archivo PNG)
    _ = plt.savefig(f"{path}/curva_dia_demanda_max_{cod_cli}.png", format='png', dpi=150)  # Puedes cambiar el formato a 'jpg', 'pdf', etc.
    
    # Cerrar la figura después de guardarla para liberar recursos
    _ = plt.close()

In [17]:
def agrupar_horas(df, columna_hora="Hora", columna_valor="Potencia_aparente_escalada"):

    # Convertir la columna Hora a tipo datetime
    df["Hora"] = pd.to_datetime(df[columna_hora], format="%H:%M")

    # Agrupar por hora y tomar el punto medio (hh:30)
    df["Hora"] = df["Hora"].dt.floor("H") + pd.Timedelta(minutes=30)

    # Recortar a HH:MM
    df["Hora"] = df["Hora"].astype("string").apply(lambda x: x.split()[1][:-3])

    # Agrupar por la nueva columna de hora y calcular el promedio de los valores
    return df.groupby("Hora")[columna_valor].apply(np.mean).reset_index()

In [18]:
def agrupar_30_min(df, columna_hora="Hora", columna_valor="Potencia_aparente_escalada"):

    # Convertir la columna Hora a tipo datetime
    df["Hora"] = pd.to_datetime(df[columna_hora], format="%H:%M")

    # Redondear hacia arriba al final del intervalo de 30 minutos
    df["Hora"] = df["Hora"] + pd.Timedelta(minutes=30)
    df["Hora"] = df["Hora"].dt.floor("H") + (df["Hora"].dt.minute // 30) * pd.Timedelta(minutes=30)

    # Recortar a HH:MM
    df["Hora"] = df["Hora"].astype("string").apply(lambda x: x.split()[1][:-3])

    # Agrupar por la nueva columna de hora y calcular el promedio de los valores
    return df.groupby("Hora")[columna_valor].apply(np.mean).reset_index()

In [19]:
def graficar_cliente(df, columna_potencia, carpeta_salida):
    """
    Genera y guarda una gráfica por cada día para un cliente.

    Parámetros:
    - df: DataFrame con las columnas 'Fecha', 'Hora' y columna_potencia.
    - columna_potencia: Nombre de la columna con los valores de potencia a graficar.
    - carpeta_salida: Ruta de la carpeta donde se guardarán las gráficas.
    """
    # Crear carpeta si no existe
    os.makedirs(carpeta_salida, exist_ok=True)
    
    # Unir Fecha y Hora en un solo datetime
    df["Datetime"] = pd.to_datetime(df["Fecha"].astype("string") + " " + df["Hora"])
    
    # Agrupar por día
    df["Fecha"] = pd.to_datetime(df["Fecha"])
    dias = df["Fecha"].unique()

    print(f"Generando {len(dias)} gráficas en la carpeta '{carpeta_salida}'...")
    start_time = time.time()

    for dia in dias:
        df_dia = df[df["Fecha"] == dia]
        dia_semana = dia.strftime("%A")  # Obtener el nombre del día (Monday, Tuesday, etc.)

        # Crear figura
        _ = plt.figure(figsize=(8, 4))
        _ = plt.plot(df_dia["Hora"], df_dia[columna_potencia], marker="o", linestyle="-")
        _ = plt.title(f"{dia_semana} - {dia.date()} Potencia Escalada")
        _ = plt.xlabel("Hora")
        _ = plt.ylabel("Potencia Escalada")
        _ = plt.xticks(df_dia["Hora"].values[::4], rotation=45)
        _ = plt.yticks(np.arange(0, 1.1, 0.1))
        _ = plt.tight_layout()

        # Guardar gráfico con día de la semana en el nombre
        nombre_archivo = os.path.join(carpeta_salida, f"{dia_semana}-{dia.date()}.png")
        _ = plt.savefig(nombre_archivo, dpi=100)
        _ = plt.close()

    end_time = time.time()
    total_tiempo = end_time - start_time
    print(f"✅ Gráficas generadas en {total_tiempo:.2f} segundos.")

    return total_tiempo

### Proceso de agregación y obtención de las curvas

In [20]:
path_entregables = "../outputs"

dict_todos_los_clientes = dict_dfs_procesados_g1 | dict_dfs_procesados_g2

In [21]:
arr_clientes_graf = ['12', '1234002', '1457182', '1627579', '10020880', '90000503', '90001202', 'TEXTILES TEXSA', 'GC NOVOPAN DEL ECUADOR', 'FLEXIPLAST', 'ECOFROZ', 'IDEAL ALAMBREC', 'LOS COCOS', 'PLANTA PANIFICADORA', 'PRINTOPAC']

In [22]:
carpeta_todas_ctipos = 'curvas_diarias'
carpeta_entregables = 'entregables_por_cliente_v2'
carpeta_ctipos = 'curvas_tipo_v2'

In [51]:
#del dict_dfs_procesados_g1
#del dict_dfs_procesados_g2

registros_curvas_todas = []

for cliente, df_medicion_anual in dict_todos_los_clientes.items():

    # Lista para almacenar todos los valores de la curva tipo del cliente
    registros_curva_cliente = []
    
    # Verificar si a ese cliente se le tienen que graficar todos los días
    #if cliente in arr_clientes_graf:
        #dir_dias_todos_cliente = fr"{path_entregables}/{carpeta_todas_ctipos}/{cliente}"
        #graficar_cliente(df_medicion_anual, 'Potencia_aparente_escalada', dir_dias_todos_cliente)

    # Crear el directorio para los entregables (Si no existe)
    dir_entregables_cli = fr"{path_entregables}/{carpeta_entregables}/{cliente}"
    os.makedirs(os.path.normpath(dir_entregables_cli), exist_ok=True)

    # Crear el directorio para guardar únicamente las curvas tipo
    dir_ctipos = fr"{path_entregables}/{carpeta_ctipos}"
    os.makedirs(os.path.normpath(dir_ctipos), exist_ok=True)
    
    # Generar el archivo con los datos de la curva tipo
    df_curva_tipo = obtener_coords_curva_tipo(df_medicion_anual)
    #df_curva_tipo = agrupar_30_min(df_curva_tipo)
    df_curva_tipo.to_csv(f"{dir_entregables_cli}/datos_curva_tipo_{cliente}.csv", index=False)

    # Guardar los datos en una lista
    registros_curva_cliente.append(cliente)
    for valor in df_curva_tipo["Potencia_aparente_escalada"].values:
        registros_curva_cliente.append(valor)

    # Generar el archivo con los datos de la curva del día que hubo la demanda máxima
    fecha_max_dem, df_curva_dia_dem_max = obtener_coords_dia_demanda_max(df_medicion_anual)
    df_curva_dia_dem_max = df_curva_dia_dem_max.sort_values(by="Hora", ascending=True)
    df_curva_dia_dem_max.to_csv(f"{dir_entregables_cli}/datos_curva_dia_demanda_max.csv", index=False)

    # Generar la gráfica de la curva tipo
    graficar_curva_tipo(df_curva_tipo, cliente, dir_entregables_cli, carpeta_ctipos)

    # Generar la gráfica de la curva del día de demanda máxima
    graficar_dia_max_demanda(df_curva_dia_dem_max, cliente, dir_entregables_cli, str(fecha_max_dem).split()[0])
    
    # Filtrar el DataFrame para ese día
    df_dia_max = df_medicion_anual[df_medicion_anual["Fecha"].dt.date == fecha_max_dem.date()]

    # Generar un archivo plano con la demanda máximo y mínima
    open(rf'{dir_entregables_cli}/Potencia_max_min.txt', 'w')\
        .write(f'Pot_aparente_max: {df_dia_max["Potencia_aparente"].max()}\nPot_aparente_min: {df_dia_max["Potencia_aparente"].min()}')

    # Guardar la lista con los registros de un cliente en otra lista
    registros_curvas_todas.append(registros_curva_cliente)

columnas_df_todas_las_curvas = ["Cliente"]
columnas_df_todas_las_curvas.extend(df_curva_tipo["Hora"].values)

### 4. Cargar los datos a MongoDB (únicamente los de curvas tipo)

In [41]:
del dict_todos_los_clientes

In [24]:
df_registros_todas_las_curvas = pd.DataFrame(registros_curvas_todas)
df_registros_todas_las_curvas.columns = columnas_df_todas_las_curvas
df_registros_todas_las_curvas

,Cliente,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,02:00,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
0,1699791,0.006641,0.006721,0.006521,0.005803,0.006295,0.006814,0.006193,0.006814,0.006778,...,0.007477,0.007086,0.008579,0.007074,0.007934,0.006692,0.006564,0.007299,0.006337,0.007301
1,1132611,0.081502,0.066426,0.052880,0.050020,0.048880,0.049070,0.045808,0.045223,0.045371,...,0.169567,0.168023,0.160208,0.143137,0.127647,0.127833,0.128325,0.122728,0.121205,0.123844
2,90001164,0.401711,0.384288,0.361851,0.366764,0.380571,0.394248,0.395500,0.397604,0.413474,...,0.362813,0.323424,0.331992,0.408344,0.434022,0.442570,0.448411,0.444237,0.445044,0.443231
3,10016608,0.854030,0.860039,0.830305,0.837424,0.835443,0.828038,0.830388,0.839721,0.834598,...,0.864170,0.826169,0.786728,0.777121,0.819110,0.813144,0.852251,0.857946,0.860029,0.867627
4,90002274,0.780804,0.824971,0.831780,0.820684,0.824391,0.835201,0.801928,0.818192,0.805883,...,0.663551,0.599305,0.555315,0.552093,0.568132,0.581777,0.623256,0.693446,0.736397,0.793248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384,SIGMAPLAST,0.629019,0.630376,0.633878,0.628827,0.628891,0.645276,0.620338,0.625429,0.605990,...,0.714449,0.731757,0.722260,0.691284,0.708349,0.708756,0.718092,0.726780,0.729823,0.736136
385,SINTOFIL,0.570382,0.585197,0.573659,0.597055,0.576146,0.579117,0.577594,0.575732,0.573957,...,0.718854,0.682750,0.618568,0.668834,0.652609,0.609928,0.658959,0.648654,0.672968,0.686380
386,SOCIEDAD INDUSTRIAL RELI CYRANO,0.507827,0.548428,0.602141,0.641924,0.676887,0.664849,0.604600,0.519819,0.460527,...,0.143328,0.236751,0.309762,0.366178,0.461476,0.496907,0.532044,0.538810,0.510710,0.507754
387,TEXTILES TEXSA,0.649134,0.655870,0.673144,0.686085,0.654700,0.630197,0.620000,0.622630,0.569756,...,0.658813,0.661239,0.658601,0.646816,0.667399,0.659044,0.700041,0.704060,0.703403,0.681430


In [25]:
### Verificar valores nulos
df_registros_todas_las_curvas[df_registros_todas_las_curvas.isna().any(axis=1)]

,Cliente,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,02:00,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
234,90000662,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
### Verificar duplicados
df_registros_todas_las_curvas["Cliente"] = df_registros_todas_las_curvas["Cliente"].astype("string")
df_registros_todas_las_curvas["Cliente"] = df_registros_todas_las_curvas["Cliente"].apply(lambda x: x[1:] if x.startswith('0') else x)
df_registros_todas_las_curvas[df_registros_todas_las_curvas["Cliente"].duplicated(keep=False)]

,Cliente,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,02:00,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
29,90000664,0.018723,0.019424,0.01805,0.01641,0.016649,0.013683,0.012931,0.012956,0.013109,...,0.085344,0.079758,0.074629,0.065755,0.052652,0.04661,0.044091,0.035944,0.026084,0.020125
311,90000664,0.018723,0.019424,0.01805,0.01641,0.016649,0.013683,0.012931,0.012956,0.013109,...,0.085344,0.079758,0.074629,0.065755,0.052652,0.04661,0.044091,0.035944,0.026084,0.020125


In [27]:
### Excluir filas con valores nulos o filas duplicadas
excluir = ['90000662',
           '090000664']
df_registros_todas_las_curvas = df_registros_todas_las_curvas[~df_registros_todas_las_curvas["Cliente"].isin(excluir)]
df_registros_todas_las_curvas = df_registros_todas_las_curvas.drop_duplicates(subset="Cliente")

In [28]:
df_registros_todas_las_curvas.shape

(387, 97)

In [29]:
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from pymongo import MongoClient

def obtener_cliente_db():
    # Cargar las variables del archivo .env
    load_dotenv()

    # Obtener las credenciales y el cluster de MongoDB
    username = os.getenv("DB_USER_T")
    password = os.getenv("DB_PASS_T")
    cluster = os.getenv("DB_CLUSTER_T")

    # Construir la uri con las credenciales
    uri = f"mongodb+srv://{username}:{password}@{cluster.lower()}.gypwd.mongodb.net/?retryWrites=true&w=majority&appName={cluster}"

    # Crear un cliente y conectarlo al servidor
    client = MongoClient(uri, 
                        server_api=ServerApi('1'),
                        connectTimeoutMS=60000,
                        socketTimeoutMS=60000,
                        serverSelectionTimeoutMS=60000,
                        tls=True)

    # Si el cliente existe, retornarlo
    if client is not None:
        return client

In [30]:
# Obtener cliente para conectar a la db
db_cliente = obtener_cliente_db()

# Convertir el dataframe a diccionario
datos_insertar = df_registros_todas_las_curvas.to_dict(orient='records')

# Eliminar cualquier documento existente en la colección
db_cliente.CurvasTipo.CurvasTipo_15m_v2.delete_many({})

# Creación de índice para el campo de la fecha
db_cliente.CurvasTipo.CurvasTipo_15m_v2.create_index([("Cliente", 1)])

# Insertar en la base de datos
db_cliente.CurvasTipo.CurvasTipo_15m_v2.insert_many(datos_insertar)

InsertManyResult([ObjectId('687154c6b4f8ddf88278aaa0'), ObjectId('687154c6b4f8ddf88278aaa1'), ObjectId('687154c6b4f8ddf88278aaa2'), ObjectId('687154c6b4f8ddf88278aaa3'), ObjectId('687154c6b4f8ddf88278aaa4'), ObjectId('687154c6b4f8ddf88278aaa5'), ObjectId('687154c6b4f8ddf88278aaa6'), ObjectId('687154c6b4f8ddf88278aaa7'), ObjectId('687154c6b4f8ddf88278aaa8'), ObjectId('687154c6b4f8ddf88278aaa9'), ObjectId('687154c6b4f8ddf88278aaaa'), ObjectId('687154c6b4f8ddf88278aaab'), ObjectId('687154c6b4f8ddf88278aaac'), ObjectId('687154c6b4f8ddf88278aaad'), ObjectId('687154c6b4f8ddf88278aaae'), ObjectId('687154c6b4f8ddf88278aaaf'), ObjectId('687154c6b4f8ddf88278aab0'), ObjectId('687154c6b4f8ddf88278aab1'), ObjectId('687154c6b4f8ddf88278aab2'), ObjectId('687154c6b4f8ddf88278aab3'), ObjectId('687154c6b4f8ddf88278aab4'), ObjectId('687154c6b4f8ddf88278aab5'), ObjectId('687154c6b4f8ddf88278aab6'), ObjectId('687154c6b4f8ddf88278aab7'), ObjectId('687154c6b4f8ddf88278aab8'), ObjectId('687154c6b4f8ddf88278aa